In [105]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from ast import literal_eval
import json

In [106]:
train = "../Datasets/spaced_catslu/train.csv"
vad = "../Datasets/spaced_catslu/development.csv"
test = "../Datasets/spaced_catslu/test.csv"

In [107]:
train = pd.read_csv(train, header=0)
vad = pd.read_csv(vad, header=0)
test = pd.read_csv(test, header=0)

In [108]:
train['transcript'][0]

'n a l ts a ɪ n ʁ i m iː ʐ ə ʁ w a lʲ ə v a ə'

In [109]:
def build_phone_vocab(df_all, temp, cnt_):
    # Convert string of list to string, i.e., "['a', 'b']" to ['a', 'b']
    length = len(train['transcript'])
    cnt = cnt_
    for i in range(length):
        if type(train['transcript'][i]) != str:
            continue
        for phone in train['transcript'][i]:
            if phone not in temp:
                temp[phone] = cnt
                cnt += 1
    return temp, cnt

In [110]:
def save_phone_idx(phone2idx, save_dir="../memory_Chinese"):
    with open(os.path.join(save_dir, "phone_idx.json"), 'w', encoding='utf-8') as f:
        json.dump(phone2idx, f)
    print("phone_idx.json saved in {}".format(os.path.abspath(save_dir)))

In [111]:
phone2idx, cnt = build_phone_vocab(train, dict(), 0)
phone2idx, cnt = build_phone_vocab(vad, phone2idx, cnt)
phone2idx, cnt = build_phone_vocab(test, phone2idx,cnt)

In [112]:
save_phone_idx(phone2idx)

phone_idx.json saved in C:\Users\dennisxchen\Desktop\SLU-M-BERT-main\memory_Chinese


In [113]:
len(phone2idx)

78

### Save data as "train_x.npy" and "train_y.npy"

In [114]:
def load_phone_idx(file_path="../memory_Chinese/phone_idx.json"):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [115]:
def read_data(filename):
    return pd.read_csv(filename)

In [116]:
def get_xy(df, phone2idx):
    length = len(df)
    labels = df['label']
    labels_return = list()
    df = df['transcript']
    X_all = list()
    for i in range(length):
        if type(df[i]) != str:
            continue
        temp = list()
        for sample in df[i]:
            x = [phone2idx[phone] for phone in sample]
            temp.append(x)
        X_all.append(temp)
        labels_return.append(labels[i])
    X_all = np.array(X_all, dtype=object)
    return X_all, labels_return

In [117]:
phone2idx = load_phone_idx("../memory_Chinese/phone_idx.json")

In [118]:
train_x, train_labels = get_xy(train, phone2idx)
vad_x, vad_labels = get_xy(vad, phone2idx)
test_x, test_labels = get_xy(test, phone2idx)

In [119]:
def save_numpy_data(filename, data):
    with open(filename, 'wb') as f:
        np.save(f, data)

In [120]:
save_numpy_data("../memory_Chinese/train_x.npy", train_x)
save_numpy_data("../memory_Chinese/train_labels.npy", train_labels)
save_numpy_data("../memory_Chinese/vad_x.npy", vad_x)
save_numpy_data("../memory_Chinese/vad_labels.npy", vad_labels)
save_numpy_data("../memory_Chinese/test_x.npy", test_x)
save_numpy_data("../memory_Chinese/test_labels.npy", test_labels)